# A) Data Preparation 🔍✍
1) Load either student-mat.csv, student-por.csv, or a merged view.
2) Validate schema & types; check duplicates.
3) Assess and handle missing values and outliers (justify method).
4) Write a short “Data Quality Report”.

In [2]:
# Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, Dropdown
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import json

# Make sure artifacts folder exists
os.makedirs("../artifacts", exist_ok=True)


In [3]:
# Load datasets (raw)
mat_path = "../data/student-mat.csv"
por_path = "../data/student-por.csv"

df_mat = pd.read_csv(mat_path, sep=";")
df_por = pd.read_csv(por_path, sep=";")
df_merged = pd.concat([df_mat, df_por], axis=0).reset_index(drop=True)

datasets = {
    "student-mat": df_mat,
    "student-por": df_por,
    "merged": df_merged
}

print("Datasets loaded successfully!")


Datasets loaded successfully!


In [ ]:
# Dataset selection
def choose_dataset(dataset_name):
    global df, active_dataset_name
    df = datasets[dataset_name]
    active_dataset_name = dataset_name
    print(f"Selected dataset: {dataset_name}")
    print("Shape:", df.shape)
    display(df.head())

interact(
    choose_dataset,
    dataset_name=Dropdown(
        options=["student-mat", "student-por", "merged"], 
        description="Choose dataset:"
    )
)


interactive(children=(Dropdown(description='Choose dataset:', options=('student-mat', 'student-por', 'merged')…

<function __main__.choose_dataset(dataset_name)>

In [4]:
# Schema & dtypes validation
print("Dataset info:")
print(df.info())


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      1044 non-null   object
 1   sex         1044 non-null   object
 2   age         1044 non-null   int64 
 3   address     1044 non-null   object
 4   famsize     1044 non-null   object
 5   Pstatus     1044 non-null   object
 6   Medu        1044 non-null   int64 
 7   Fedu        1044 non-null   int64 
 8   Mjob        1044 non-null   object
 9   Fjob        1044 non-null   object
 10  reason      1044 non-null   object
 11  guardian    1044 non-null   object
 12  traveltime  1044 non-null   int64 
 13  studytime   1044 non-null   int64 
 14  failures    1044 non-null   int64 
 15  schoolsup   1044 non-null   object
 16  famsup      1044 non-null   object
 17  paid        1044 non-null   object
 18  activities  1044 non-null   object
 19  nursery     1044 non-null   object

In [5]:
# Check duplicates
print("Number of duplicate rows:", df.duplicated().sum())


Number of duplicate rows: 0


In [6]:
# Check missing values
print("Missing values:")
print(df.isnull().sum().sort_values(ascending=False).head())


Missing values:
school      0
paid        0
G2          0
G1          0
absences    0
dtype: int64


In [7]:
# Outlier detection (IQR)
numeric_cols = df.select_dtypes(include=[np.number]).columns

outlier_report = {}
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    outlier_report[col] = outliers

pd.DataFrame.from_dict(outlier_report, orient='index', columns=['Outlier_Count'])


,Outlier_Count
age,2
Medu,0
Fedu,0
traveltime,24
studytime,62
failures,183
famrel,77
freetime,64
goout,0
Dalc,52


In [8]:
# Save cleaned version of selected dataset
df.to_csv(f"../artifacts/{active_dataset_name}-clean.csv", index=False)

# Save active dataset name to config
active_ds = {"active_dataset": active_dataset_name}
with open("../artifacts/active_dataset.json", "w") as f:
    json.dump(active_ds, f)

print(f"Saved: ../artifacts/{active_dataset_name}-clean.csv")
print(f"Active dataset set to: {active_dataset_name}")


Saved: ../artifacts/merged-clean.csv
Active dataset set to: merged


# Data Quality Report 📃✅


## Datasets Loaded
- **Math:** 395 rows, 33 columns  
- **Portuguese:** 649 rows, 33 columns  
- **Merged:** 1044 rows, 33 columns  

## Schema & Types
- All datasets share identical schema.  
- Features include demographics (school, sex, age, address...), family background, study habits, absences, and grades (G1, G2, G3).  

## Duplicates
- Math: **0 duplicates**  
- Portuguese: **0 duplicates**  
- Merged: **0 duplicates**  

## Missing Values
- **No missing values** detected in any dataset (✓).  

## Outliers (IQR method, numeric only)

### Math (student-mat)
- High outliers:  
  - **failures (83 students)** → many students with extreme failure counts.  
  - **absences (15 students)** → extreme truancy.  
  - **studytime (27), traveltime (8), freetime (19)** → some extreme behaviors.  
  - **alcohol use (Dalc=18)** → weekend alcohol also shows moderate outliers.  
  - **grades (G2=13)** but G1/G3 within expected ranges.  
- Other variables show very limited or no strong outliers.  

### Portuguese (student-por)
- High outliers:  
  - **failures (100 students)** → extreme failure counts.  
  - **studytime (35), traveltime (16), freetime (45), famrel (51)** → multiple behavioral outliers.  
  - **absences (21 students)** → truancy cases.  
  - **grades (G1=16, G2=25, G3=16)** → some grade distribution anomalies, but still within 0–20 scale.  
  - **alcohol use (Dalc=34)**.  

### Merged
- Outlier counts naturally increase since datasets are combined:  
  - **failures (183 students)** → most significant outlier dimension.  
  - **studytime (62), freetime (64), famrel (77)**.  
  - **absences (54 students)** → extreme truancy across merged data.  
  - **grades (G2=20, G3=54)** → grade anomalies increase with combined set.  
  - **alcohol use (Dalc=52)**.  

## Interpretation
- The majority of features are within reasonable ranges, but **failures, absences, studytime, freetime, and alcohol consumption** show meaningful outliers that may reflect real risky behaviors rather than pure noise.  
- Grades (G1, G2, G3) are bounded 0–20 but still flagged when distributions are skewed.  


# B) Data Transformation
1) Encoding

2) Scaling

3) Feature Engineering 

4) Leakage note

In [ ]:
# Identify categorical and numeric features

categorical_features = [
    "school", "sex", "address", "famsize", "Pstatus",
    "Mjob", "Fjob", "reason", "guardian", "schoolsup",
    "famsup", "paid", "activities", "nursery", "higher",
    "internet", "romantic"
]

numeric_features = [col for col in df.columns if col not in categorical_features]

print("Categorical features:", categorical_features)
print("Numeric features:", numeric_features)


Categorical features: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
Numeric features: ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']


In [ ]:
# OneHot Encoding
encoder = OneHotEncoder(drop="first", sparse_output=False)
encoded = encoder.fit_transform(df[categorical_features])

encoded_cols = encoder.get_feature_names_out(categorical_features)
df_encoded = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)

df_numeric = df.drop(columns=categorical_features)
df_transformed = pd.concat([df_numeric, df_encoded], axis=1)

print("Transformed shape:", df_transformed.shape)
display(df_transformed.head())


Transformed shape: (1044, 42)


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,3,4,3,2,2,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


In [ ]:
# Scaling numeric features
scaler = StandardScaler()
scaled = scaler.fit_transform(df_transformed[numeric_features])

df_scaled = df_transformed.copy()
df_scaled[numeric_features] = scaled

print("Shape after scaling:", df_scaled.shape)
display(df_scaled.head())


Shape after scaling: (1044, 42)


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,1.027889,1.242077,1.466302,0.652210,0.035606,-0.403106,0.068788,-0.195099,0.732511,-0.542374,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.221035,-1.426089,-1.262431,-0.715074,0.035606,-0.403106,1.140653,-0.195099,-0.135527,-0.542374,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-1.392674,-1.426089,-1.262431,-0.715074,0.035606,4.171268,0.068788,-0.195099,-1.003566,0.554987,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
3,-1.392674,1.242077,-0.352853,-0.715074,1.234713,-0.403106,-1.003076,-1.165019,-1.003566,-0.542374,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,-0.585820,0.352689,0.556724,-0.715074,0.035606,-0.403106,0.068788,-0.195099,-1.003566,-0.542374,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


In [14]:
# Feature Engineering
df_scaled["avg_grade"] = df_scaled[["G1", "G2", "G3"]].mean(axis=1)
df_scaled["attendance"] = df_scaled["absences"].apply(lambda x: 1 if x < 5 else 0)
df_scaled["pass_binary"] = (df_scaled["G3"] >= 10).astype(int)

# Risk tiers (0=Fail, 1=Medium, 2=High)
def risk_level(g3):
    if g3 < 10:
        return 0
    elif g3 < 15:
        return 1
    else:
        return 2
df_scaled["risk_tier"] = df_scaled["G3"].apply(risk_level)

print("Feature engineering added.")
display(df_scaled[["G1","G2","G3","avg_grade","attendance","pass_binary","risk_tier"]].head())


Feature engineering added.


,G1,G2,G3,avg_grade,attendance,pass_binary,risk_tier
0,-2.083727,-1.597738,-1.382871,-1.688112,1,0,0
1,-2.083727,-1.902291,-1.382871,-1.789630,1,0,0
2,-1.413029,-0.988631,-0.347391,-0.916350,1,0,0
3,1.269766,0.838688,0.946958,1.018471,1,0,0
4,-1.748378,-0.379525,-0.347391,-0.825098,1,0,0


In [15]:
# Leakage Variants
# Variant A: With G1, G2
df_with_g1g2 = df_scaled.copy()

# Variant B: Without G1, G2
df_without_g1g2 = df_scaled.drop(columns=["G1","G2"])

print(f"Variant A shape (with G1, G2):", df_with_g1g2.shape)
print(f"Variant B shape (without G1, G2):", df_without_g1g2.shape)


Variant A shape (with G1, G2): (1044, 46)
Variant B shape (without G1, G2): (1044, 44)


In [20]:
# Save datasets
df_with_g1g2.to_csv(f"../artifacts/{active_dataset_name}_with_g1g2.csv", index=False)
df_without_g1g2.to_csv(f"../artifacts/{active_dataset_name}_without_g1g2.csv", index=False)

print("Saved datasets to artifacts/")


Saved datasets to artifacts/


In [ ]:
# Define variants
variants = {
    "with_g1g2": f"../artifacts/{active_dataset_name}_with_g1g2.csv",
    "without_g1g2": f"../artifacts/{active_dataset_name}_without_g1g2.csv"
}

print("Available variants:")
for k,v in variants.items():
    print(f" - {k}: {v}")


Available variants:
 - with_g1g2: ../artifacts/merged_with_g1g2.csv
 - without_g1g2: ../artifacts/merged_without_g1g2.csv


In [18]:
# Variant selection (with dropdown)
from ipywidgets import interact, Dropdown

def choose_variant(variant_name):
    global df_variant, active_variant_name
    df_variant = pd.read_csv(variants[variant_name])
    active_variant_name = variant_name
    
    # Save active variant name to config
    active_var = {"active_variant": active_variant_name}
    with open("../artifacts/active_variant.json", "w") as f:
        json.dump(active_var, f)
    
    print(f"Selected variant: {variant_name}")
    print("Shape:", df_variant.shape)
    display(df_variant.head())

interact(
    choose_variant,
    variant_name=Dropdown(
        options=list(variants.keys()), 
        description="Choose variant:"
    )
)


interactive(children=(Dropdown(description='Choose variant:', options=('with_g1g2', 'without_g1g2'), value='wi…

<function __main__.choose_variant(variant_name)>

In [ ]:
# Final dataset after encoding ---
import pandas as pd

# Concat encoded categorical features مع الـ numeric
df_final = pd.concat(
    [pd.DataFrame(encoded, columns=encoded_cols, index=df.index), df[numeric_features]],
    axis=1
)

print("✅ Final encoded dataset ready!")
print("Shape:", df_final.shape)
df_final.head()


✅ Final encoded dataset ready!
Shape: (1044, 42)


,school_MS,sex_M,address_U,famsize_LE3,Pstatus_T,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_health,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,3,4,1,1,3,6,5,6,6
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5,3,3,1,1,3,4,5,5,6
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4,3,2,2,3,3,10,7,8,10
3,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,3,2,2,1,1,5,2,15,14,15
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,4,3,2,1,2,5,4,6,10,10


In [24]:
# Save the final dataset
processed_path = f"../artifacts/{active_dataset_name}-processed.csv"
df_final.to_csv(processed_path, index=False)

# Update config
config = {
    "active_dataset": active_dataset_name,
    "clean_path": f"../artifacts/{active_dataset_name}-clean.csv",
    "processed_path": processed_path
}

with open("../artifacts/active_dataset.json", "w") as f:
    json.dump(config, f)

print(f"📂 Saved processed dataset: {processed_path}")


📂 Saved processed dataset: ../artifacts/merged-processed.csv


In [26]:
df_final.shape

(1044, 42)